In [7]:
import pygame.midi
import time

pygame.midi.init()
player = pygame.midi.Output(0)
player.set_instrument(48) # https://pjb.com.au/muscript/gm.html
player.note_on(64, 127) # note_on(note, velocity=None, channel = 0) https://en.scratch-wiki.info/wiki/MIDI_Notes
time.sleep(2)
player.note_off(64, 127)
del player
pygame.midi.quit()

In [11]:
# from https://gist.github.com/karpathy/d4dee566867f8291f086

"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

nruns = 1000

# data I/O
data = open('notes0.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
#while True:
while n<nruns:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print '----\n %s \n----' % (txt, )

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 752 characters, 10 unique.
----
 90207949861874227
94088
9099
26

018257009
94915
60951195624046724064955
979802451566917
57654217
44

51142528525
681775406
51451241972
198805206816851296
71497596
862995

596769528


00962182579198
8 
----
iter 0, loss: 57.564630
----
 667
29

99
6
6
46




67762
8
966497762
7
24
7565

4
6277


662

69728784
7


607
427
6

67


2
6
779
77056
67
7
69
947
79

269


57797777267

666
4467789804649

74762
76
62094748
1167177794766
89
7
4 
----
iter 100, loss: 57.960934
----
 62
79
62
74
62
64
62
79
62
74
72
70
64
74
79
74
62164
70
70
64
7

76
60
69
79
66
77
60
66
69
64
62
67
8

64
69
67
60
74
64
64
61
67
00
69
75
47
61
64
55
6

69
62
56
67
59
74
74
69
68
5

66
76
71
60
76 
----
iter 200, loss: 55.407554
----
 4
74
64
74
66
70
62
72
79
64
66
70
64
67
67
74
76
64
64
67
66
62
64
67
74
72
64
79
57
76
64
67
64
72
66
69262
64
69
66
72
69
64
59
42
67
64
74
67
79
76
69
74
70
64
79
69
72
67
54
89
64
67
74
79
72
77
 
----
iter 300, loss: 52.872806
----
 
62


In [3]:
import pygame.midi
import time

pygame.midi.init()
player = pygame.midi.Output(0)
player.set_instrument(0) # https://pjb.com.au/muscript/gm.html
notel = [72, 64, 64, 59, 60, 62, 67, 79, 72, 74, 77, 67, 62, 72,79,79,64,64,69,79,67,67,64,64,62,64,64,62,72,79,62,60,67,67,69,69,69,69,64,69,62,62,60,59,59,62,67,67,74,72,72,72,81,60,67,67,76,76,62,60,74,64,67,67,76]
for n in notel:
    player.note_on(n, 127) # note_on(note, velocity=None, channel = 0) https://en.scratch-wiki.info/wiki/MIDI_Notes
    time.sleep(0.2)
    player.note_off(n, 127)
del player
pygame.midi.quit()

In [8]:
import mido

#outport = mido.open_output()
niter = 0
for msg in mido.MidiFile('bach_846.mid').play():
    #outport.send(msg)
    #print msg.bytes()
    #print str(msg)
    thismsg = str(msg).split(" ") # ['note_on', 'channel=0', 'note=60', 'velocity=61', 'time=0.00168918958333']
    if thismsg[0] == 'note_on':
        print thismsg[2][5:]
    niter += 1
    if niter >= 300:
        break

60
64
67
67
72
72
76
76
67
67
72
72
76
76
64
60
60
64
67
67
72
72
76
76
67
67
72
72
76
76
64
60
60
62
69
69
74
74
77
77
69
69
74
74
77
77
62
60
60
62
69
69
74
74
77
77
69
69
74
74
77
77
62
60
59
62
67
67
74
74
77
77
67
67
74
74
77
77
62
59
59
62
67
67
74
74
77
77
67
67
74
74
77
77
62
59
60
64
67
67
72
72
76
76
67
67
72
72
76
76
64
60
60
64
67
67
72
72
76
76
67
67
72
72
76
76
64
60
60
64
69
69
76
76
81
81
69
69
76
76
81
81
64
60
60
64
69
69
76
76
81
81
69
69
76
76
81
81
64
60
60
62
66
66
69
69
74
74
66
66
69
69
74
74
62
60
60
62
66
66
69
69
74
74
66
66
69
69
74
74
62
60
59
62
67
67
74
74
79
79
67
67
74
74
79
79
62
59
59
62
67
67
74
74
79
79
67
67
74
74
79
79
62
59
59
60
64
64
67
67
72
72
64
64
67
67
72
72
60
59
59
60
64
64
67
67
72
72
64
64
67
